# VM 使用 LLaMA-Factory Docker 部署 Phi-4 模型完整教程

本 Jupyter Notebook 提供了在虚拟机（VM）上使用 LLaMA-Factory Docker 方式部署 Phi-4 模型的完整指南。本教程涵盖了从环境配置到模型部署和测试的所有步骤。

## 目录
1. [概述](#概述)
2. [环境要求](#环境要求)
3. [VM 环境准备](#vm-环境准备)
4. [安装依赖](#安装依赖)
5. [下载模型](#下载模型)
6. [部署模型](#部署模型)
7. [测试验证](#测试验证)
8. [性能优化](#性能优化)
9. [常见问题](#常见问题)

## 概述

Phi-4 是微软发布的一个高效的小型语言模型，本教程将指导您如何在虚拟机（VM）上完整部署 Phi-4 模型，包括环境配置、模型下载、部署和测试等步骤。

## 环境要求

### 硬件要求
- **CPU**: 至少 8 核心（推荐 16 核心或以上）
- **内存**: 最少 16GB RAM（推荐 32GB 或以上）
- **存储**: 至少 50GB 可用空间
- **GPU**: 可选，但强烈推荐使用 NVIDIA GPU（至少 8GB 显存）

### 软件要求
- **操作系统**: Ubuntu 20.04/22.04 LTS 或 CentOS 7/8
- **Python**: 3.8 或更高版本
- **CUDA**: 11.8 或更高版本（如果使用 GPU）

## 环境准备

### 定义环境变量

我们需要设置一些环境变量来管理我们的资源：

In [ ]:
import os
import random

# 设置环境变量
os.environ["RESOURCE_GROUP"] = "lenovo_vm_rg"
os.environ["LOCATION"] = "eastus2"
os.environ['VM_NAME'] = "llama3_1_8b-vm"
os.environ['VM_SIZE'] = "Standard_D4s_v3"

# 显示配置
print(f"Resource Group: {os.environ['RESOURCE_GROUP']}")
print(f"Location: {os.environ['LOCATION']}")
print(f"VM Name: {os.environ['VM_NAME']}")
print(f"VM Size: {os.environ['VM_SIZE']}")


## VM 环境准备

### 1. 创建资源组

以下使用 Azure CLI 创建 VM（需要先安装和配置 Azure CLI）：

In [ ]:
%%bash
az group create \
    --name $RESOURCE_GROUP \
    --location $LOCATION

### 2. 创建虚拟机

以下使用 Azure CLI 创建 VM（需要先安装和配置 Azure CLI）：

In [1]:
# 使用 Azure CLI 创建 VM
!az vm create \
  --resource-group $RESOURCE_GROUP \
  --name $VM_NAME \
  --image Ubuntu2204 \
  --size $VM_SIZE \
  --admin-username azureuser \
  --generate-ssh-keys

ERROR: (ResourceGroupNotFound) Resource group 'test-vm-rg' could not be found.
Code: ResourceGroupNotFound
Message: Resource group 'test-vm-rg' could not be found.


### 2. 连接到 VM

获取 VM 的 IP 地址后，使用 SSH 连接：

In [ ]:
# 替换为您的 VM IP 地址
VM_IP="<VM-IP-ADDRESS>"
!ssh azureuser@$VM_IP

### 3. 更新系统

运行以下命令更新系统并安装基础工具：

In [ ]:
# 更新包列表和系统
!sudo apt update && sudo apt upgrade -y

# 安装基础工具
!sudo apt install -y build-essential git wget curl

## 安装依赖

### 1. 安装 Docker 和 Docker Compose

In [ ]:
# 安装 Docker
!curl -fsSL https://get.docker.com -o get-docker.sh
!sudo sh get-docker.sh

# 将当前用户添加到 docker 组
!sudo usermod -aG docker $USER

# 安装 Docker Compose
!sudo curl -L "https://github.com/docker/compose/releases/latest/download/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
!sudo chmod +x /usr/local/bin/docker-compose

# 启动 Docker 服务
!sudo systemctl start docker
!sudo systemctl enable docker

# 验证安装
!docker --version
!docker-compose --version

### 2. 安装 NVIDIA Container Toolkit（如果使用 GPU）

In [ ]:
# 添加 NVIDIA Docker 仓库
!curl -fsSL https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg
!curl -s -L https://nvidia.github.io/libnvidia-container/stable/deb/nvidia-container-toolkit.list | \
  sed 's#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g' | \
  sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list

# 更新包列表并安装
!sudo apt-get update
!sudo apt-get install -y nvidia-container-toolkit

# 配置 Docker 以使用 NVIDIA runtime
!sudo nvidia-ctk runtime configure --runtime=docker
!sudo systemctl restart docker

# 验证 GPU 支持
!docker run --rm --gpus all nvidia/cuda:11.8-base-ubuntu22.04 nvidia-smi

### 3. 克隆 LLaMA-Factory 项目

In [ ]:
# 克隆 LLaMA-Factory 项目
!git clone https://github.com/hiyouga/LLaMA-Factory.git
!cd LLaMA-Factory

# 查看项目结构
!ls -la LLaMA-Factory/

## 配置 LLaMA-Factory

### 1. 创建 Docker Compose 配置文件

我们需要创建一个 `docker-compose.yml` 文件来配置 LLaMA-Factory 服务：

In [ ]:
%%writefile LLaMA-Factory/docker-compose.yml
version: '3.8'

services:
  llamafactory:
    build:
      context: .
      dockerfile: Dockerfile
    container_name: llamafactory-phi4
    ports:
      - "7860:7860"  # Gradio Web UI
      - "8000:8000"  # API 服务
    volumes:
      - ./data:/app/data
      - ./saves:/app/saves
      - ./models:/app/models
      - ./logs:/app/logs
    environment:
      - CUDA_VISIBLE_DEVICES=0
      - HF_HOME=/app/models
      - HF_HUB_CACHE=/app/models
    deploy:
      resources:
        reservations:
          devices:
            - driver: nvidia
              count: all
              capabilities: [gpu]
    stdin_open: true
    tty: true
    command: bash

### 2. 创建模型配置文件

创建 Phi-4 模型的配置文件：

In [ ]:
%%writefile LLaMA-Factory/examples/inference/phi4_config.yaml
### model
model_name_or_path: microsoft/Phi-4
adapter_name_or_path: null

### method
finetuning_type: lora
quantization_bit: null

### ddp
ddp_timeout: 180000000

### output
output_dir: saves/Phi-4/lora/sft

### train
cutoff_len: 1024
learning_rate: 5.0e-5
num_train_epochs: 3.0
max_samples: 1000
per_device_train_batch_size: 2
gradient_accumulation_steps: 8
lr_scheduler_type: cosine
max_grad_norm: 1.0
logging_steps: 5
save_steps: 100
warmup_steps: 0
neftune_noise_alpha: null
optim: adamw_torch
resize_vocab: false
packing: false
upsampling: false

### eval
val_size: 0.1
per_device_eval_batch_size: 1
eval_strategy: steps
eval_steps: 500

## 部署模型

### 1. 构建 Docker 镜像

In [ ]:
# 进入 LLaMA-Factory 目录
%cd LLaMA-Factory

# 构建 Docker 镜像
!docker-compose build

# 或者直接使用预构建的镜像
!docker pull registry.cn-hangzhou.aliyuncs.com/modelscope-repo/llamafactory:latest

### 2. 启动容器并下载模型

In [ ]:
# 启动容器
!docker-compose up -d

# 进入容器
!docker exec -it llamafactory-phi4 bash

# 在容器内下载 Phi-4 模型
# 这个命令需要在容器内执行
!echo "huggingface-cli download microsoft/Phi-4 --local-dir /app/models/Phi-4"

### 3. 创建 API 服务启动脚本

创建一个脚本来启动 LLaMA-Factory 的 API 服务：

In [ ]:
%%writefile start_phi4_api.sh
#!/bin/bash

# 启动 LLaMA-Factory API 服务
docker exec -d llamafactory-phi4 llamafactory-cli api \
    --model_name_or_path microsoft/Phi-4 \
    --template default \
    --host 0.0.0.0 \
    --port 8000

echo "Phi-4 API 服务已启动在端口 8000"
echo "访问 http://localhost:8000/docs 查看 API 文档"

### 4. 创建 Web UI 启动脚本

创建一个脚本来启动 LLaMA-Factory 的 Web UI：

In [ ]:
%%writefile start_phi4_webui.sh
#!/bin/bash

# 启动 LLaMA-Factory Web UI
docker exec -d llamafactory-phi4 llamafactory-cli webui \
    --host 0.0.0.0 \
    --port 7860

echo "Phi-4 Web UI 已启动在端口 7860"
echo "访问 http://localhost:7860 使用 Web 界面"

In [ ]:
# 给脚本添加执行权限
!chmod +x start_phi4_api.sh
!chmod +x start_phi4_webui.sh

# 启动 API 服务
!./start_phi4_api.sh

# 启动 Web UI
!./start_phi4_webui.sh

## 测试验证

### 1. 检查容器状态

In [ ]:
# 检查容器运行状态
!docker ps

# 查看容器日志
!docker logs llamafactory-phi4

# 检查 GPU 使用情况（如果有 GPU）
!docker exec llamafactory-phi4 nvidia-smi

### 2. 测试 API 接口

In [ ]:
# 测试健康检查
!curl http://localhost:8000/v1/models

# 测试聊天接口
%%bash
curl -X POST "http://localhost:8000/v1/chat/completions" \
     -H "Content-Type: application/json" \
     -d '{
       "model": "microsoft/Phi-4",
       "messages": [
         {
           "role": "user",
           "content": "什么是人工智能？"
         }
       ],
       "max_tokens": 512,
       "temperature": 0.7
     }'

### 3. Python 客户端测试

创建一个 Python 客户端来测试 API：

In [ ]:
%%writefile test_phi4_client.py
import requests
import json

def test_phi4_api():
    """测试 Phi-4 API"""
    
    # API 端点
    url = "http://localhost:8000/v1/chat/completions"
    
    # 测试用例
    test_messages = [
        [{"role": "user", "content": "请解释量子计算的基本原理"}],
        [{"role": "user", "content": "写一个 Python 函数计算斐波那契数列"}],
        [{"role": "user", "content": "描述一下未来的智能城市"}]
    ]
    
    for i, messages in enumerate(test_messages, 1):
        print(f"\n=== 测试 {i} ===")
        print(f"输入: {messages[0]['content']}")
        print("-" * 50)
        
        try:
            response = requests.post(
                url,
                json={
                    "model": "microsoft/Phi-4",
                    "messages": messages,
                    "max_tokens": 512,
                    "temperature": 0.7,
                    "stream": False
                },
                timeout=60
            )
            
            if response.status_code == 200:
                result = response.json()
                content = result['choices'][0]['message']['content']
                print(f"输出: {content}")
            else:
                print(f"错误: {response.status_code} - {response.text}")
                
        except Exception as e:
            print(f"请求失败: {str(e)}")

if __name__ == "__main__":
    test_phi4_api()

In [ ]:
# 安装 requests 库
%pip install requests

# 运行测试客户端
!python test_phi4_client.py

## 性能优化

### 1. Docker 容器资源优化

为了获得更好的性能，可以调整 Docker 容器的资源配置：

In [ ]:
%%writefile LLaMA-Factory/docker-compose-optimized.yml
version: '3.8'

services:
  llamafactory:
    image: registry.cn-hangzhou.aliyuncs.com/modelscope-repo/llamafactory:latest
    container_name: llamafactory-phi4-optimized
    ports:
      - "7860:7860"
      - "8000:8000"
    volumes:
      - ./data:/app/data
      - ./saves:/app/saves
      - ./models:/app/models
      - ./logs:/app/logs
      - /tmp:/tmp  # 临时文件目录
    environment:
      - CUDA_VISIBLE_DEVICES=0
      - HF_HOME=/app/models
      - HF_HUB_CACHE=/app/models
      - PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128
      - OMP_NUM_THREADS=8
    deploy:
      resources:
        limits:
          memory: 32G
          cpus: '16'
        reservations:
          memory: 16G
          cpus: '8'
          devices:
            - driver: nvidia
              count: all
              capabilities: [gpu]
    shm_size: 8gb  # 增加共享内存
    ulimits:
      memlock: -1
      stack: 67108864
    stdin_open: true
    tty: true

### 2. 模型量化配置

为了减少内存使用，可以使用量化技术：

In [ ]:
%%writefile start_phi4_quantized.sh
#!/bin/bash

# 启动量化版本的 Phi-4 API 服务
docker exec -d llamafactory-phi4 llamafactory-cli api \
    --model_name_or_path microsoft/Phi-4 \
    --template default \
    --quantization_bit 4 \
    --host 0.0.0.0 \
    --port 8000

echo "量化版 Phi-4 API 服务已启动在端口 8000"
echo "使用 4-bit 量化以减少内存使用"

### 3. 监控和日志

In [ ]:
# 监控容器资源使用情况
!docker stats llamafactory-phi4

# 查看详细的容器信息
!docker inspect llamafactory-phi4

# 实时查看日志
!docker logs -f llamafactory-phi4

## 常见问题

### 1. Docker 容器内存不足

In [ ]:
# 增加 Docker 容器的内存限制
!docker update --memory=32g --memory-swap=64g llamafactory-phi4

# 或者在 docker-compose.yml 中设置
# deploy:
#   resources:
#     limits:
#       memory: 32G

### 2. GPU 在容器中不可用

In [ ]:
# 检查 NVIDIA Container Toolkit 是否正确安装
!docker run --rm --gpus all nvidia/cuda:11.8-base-ubuntu22.04 nvidia-smi

# 重新配置 Docker 以支持 GPU
!sudo nvidia-ctk runtime configure --runtime=docker
!sudo systemctl restart docker

# 确保 docker-compose.yml 中包含 GPU 配置
# deploy:
#   resources:
#     reservations:
#       devices:
#         - driver: nvidia
#           count: all
#           capabilities: [gpu]

In [ ]:
# 在容器内设置 Hugging Face 镜像
!docker exec llamafactory-phi4 bash -c "export HF_ENDPOINT=https://hf-mirror.com"

# 或者在 docker-compose.yml 中设置环境变量
# environment:
#   - HF_ENDPOINT=https://hf-mirror.com

# 使用代理下载（如果需要）
# environment:
#   - https_proxy=http://your-proxy:port
#   - http_proxy=http://your-proxy:port

## 总结

本教程详细介绍了如何在 VM 上使用 LLaMA-Factory Docker 方式部署 Phi-4 模型：

### 主要步骤：
1. **环境准备**：安装 Docker 和 NVIDIA Container Toolkit
2. **项目配置**：克隆 LLaMA-Factory 并配置 Docker Compose
3. **模型部署**：构建镜像、启动容器、下载模型
4. **服务启动**：启动 API 服务和 Web UI
5. **测试验证**：通过 API 和客户端测试模型功能

### 优势：
- **环境隔离**：Docker 容器提供干净的运行环境
- **易于管理**：通过 Docker Compose 统一管理服务
- **可扩展性**：支持 GPU 加速和资源优化
- **便于部署**：一键启动所有服务

### 注意事项：
- 确保 VM 有足够的内存和存储空间
- 正确配置 GPU 支持（如果使用）
- 合理设置资源限制和优化参数
- 定期监控容器状态和资源使用情况